In [9]:
import numpy as np
import pandas as pd

In [1]:
#pgm_sb_hist_lagacy
import json
import logging
import views

logging.basicConfig(
    level=logging.DEBUG,
    #level=logging.INFO, # uncomment this and comment debug above for less yelling in red
    format=views.config.LOGFMT,
)
# DATASETS is a dictionary of Dataset objects.
from views import DATASETS
# These are the building blocks of the modelling interface
from views import Ensemble, Model, Downsampling, Period
# These are model specifications from the specfiles
from views.specs.models import cm as model_specs_cm, pgm as model_specs_pgm
from views.specs.periods import get_periods, get_periods_by_name
# Utils
from views.utils import db, io, data as datautils
from views.utils.data import assign_into_df
from views.apps.pipeline.models_cm import all_cm_models_by_name
from views.apps.pipeline.models_pgm import all_pgm_models_by_name
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
dataset = views.DATASETS["cm_africa_imp_0"]
df = dataset.df
run_id = "d_2020_04_01"

[2021-01-26 15:19:59,993] - views.utils.io:107 - DEBUG - Loading YAML from /Users/brandonrose/repos/WM/views2_ensemble/OpenViEWS2/views/specs/periods/periods.yaml
[2021-01-26 15:20:00,038] - views.utils.io:107 - DEBUG - Loading YAML from /Users/brandonrose/repos/WM/views2_ensemble/OpenViEWS2/views/specs/periods/periods.yaml
[2021-01-26 15:20:00,068] - views.utils.io:65 - DEBUG - Reading parquet at /Users/brandonrose/repos/WM/views2_ensemble/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet with cols None
[2021-01-26 15:20:00,883] - views.utils.io:72 - DEBUG - Finished reading parquet from /Users/brandonrose/repos/WM/views2_ensemble/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet.


In [2]:
df

acled_count_ns  acled_count_os  acled_count_pr  \
month_id country_id                                                   
1        40                     0.0             0.0             0.0   
         41                     0.0             0.0             0.0   
         42                     0.0             0.0             0.0   
         43                     0.0             0.0             0.0   
         47                     0.0             0.0             0.0   
...                             ...             ...             ...   
612      242                    0.0             0.0             0.0   
         243                    0.0             0.0             0.0   
         244                    0.0             0.0             0.0   
         245                    0.0             0.0             0.0   
         246                    0.0             0.0             0.0   

                     acled_count_sb  acled_dummy_ns  acled_dummy_os  \
month_id country_id                                                   
1        40                     0.0               0               0   
         41                     0.0               0               0   
         42                     0.0               0               0   
         43                     0.0               0               0   
         47                     0.0               0               0   
...                             ...             ...             ...   
612      242                    0.0               0               0   
         243                    0.0               0               0   
         244                    0.0               0               0   
         245                    0.0               0               0   
         246                    0.0               0               0   

                     acled_dummy_pr  acled_dummy_sb  cdum_1  cdum_10  ...  \
month_id country_id                                                   ...   
1        40                       0               0       0        0  ...   
         41                       0               0       0        0  ...   
         42                       0               0       0        0  ...   
         43                       0               0       0        0  ...   
         47                       0               0       0        0  ...   
...                             ...             ...     ...      ...  ...   
612      242                      0               0       0        0  ...   
         243                      0               0       0        0  ...   
         244                      0               0       0        0  ...   
         245                      0               0       0        0  ...   
         246                      0               0       0        0  ...   

                     wdi_tx_val_trvl_zs_wt  wdi_vc_btl_deth  wdi_vc_idp_nwcv  \
month_id country_id                                                            
1        40                       4.124475       814.000000          23000.0   
         41                      10.417314         8.000000            500.0   
         42                       0.391773        53.857143           5000.0   
         43                      38.167939        27.000000          10000.0   
         47                      75.148042        43.000000            700.0   
...                                    ...              ...              ...   
612      242                     61.237308        81.692308           1000.0   
         243                     43.453430       207.000000             35.0   
         244                      3.752579         8.000000          36000.0   
         245                     70.229657       246.000000          84000.0   
         246                      0.711187       536.000000         259000.0   

                     wdi_vc_idp_nwds  wdi_vc_idp_tocv  wdi_vc_ihr_psrc_fe_p5  \
month_id country_id                               

In [3]:
corr_matrix = df.corr().abs()

In [32]:
corr_matrix.loc['vdem_v2x_libdem','vdem_v2x_polyarchy']

0.9596629913600213

In [6]:

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)

sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))

In [12]:
sol = pd.DataFrame(sol).reset_index()

In [17]:
sol.rename(columns={'level_0': 'feature', 'level_1': 'correlated_feature', 0: 'correlation'}, inplace=True)

In [50]:
feats_of_interest = sol[sol['feature'].isin(['vdem_v2x_libdem', 'wdi_ny_gdp_pcap_pp_kd', 'wdi_sp_dyn_imrt_in','wdi_ag_prd_food_xd'])]

In [62]:
feats_final = feats_of_interest[feats_of_interest['correlation']>=0.8].sort_values(['feature','correlation'],ascending=False)

In [63]:
feats_final.head()

,feature,correlated_feature,correlation
232,wdi_sp_dyn_imrt_in,wdi_sp_dyn_imrt_ma_in,0.987715
3439,wdi_sp_dyn_imrt_in,wdi_sp_dyn_le00_fe_in,0.897088
3736,wdi_sp_dyn_imrt_in,wdi_sp_dyn_le00_in,0.891880
4474,wdi_sp_dyn_imrt_in,wdi_sp_dyn_le00_ma_in,0.879161
8628,wdi_sp_dyn_imrt_in,wdi_sp_dyn_to65_fe_zs,0.823222


In [65]:
mapping = {}

for kk, vv in feats_final.iterrows():
    if vv.feature not in mapping:
        mapping[vv.feature] = [vv.correlated_feature]
    else:
        mapping[vv.feature].append(vv.correlated_feature)

In [66]:
mapping

{'wdi_sp_dyn_imrt_in': ['wdi_sp_dyn_imrt_ma_in',
  'wdi_sp_dyn_le00_fe_in',
  'wdi_sp_dyn_le00_in',
  'wdi_sp_dyn_le00_ma_in',
  'wdi_sp_dyn_to65_fe_zs',
  'wdi_sp_dyn_tfrt_in'],
 'wdi_ny_gdp_pcap_pp_kd': ['wdi_ny_gnp_pcap_pp_cd',
  'wdi_ny_gnp_pcap_pp_kd',
  'wdi_sl_gdp_pcap_em_kd',
  'wdi_ny_gnp_pcap_cd',
  'wdi_ny_gnp_pcap_kd',
  'wdi_sh_xpd_chex_pp_cd',
  'wdi_sh_xpd_pvtd_pp_cd',
  'wdi_sh_xpd_oopc_pc_cd',
  'wdi_sh_xpd_ghed_pp_cd',
  'wdi_sh_xpd_pvtd_pc_cd',
  'wdi_si_pov_umic',
  'wdi_sh_xpd_chex_pc_cd',
  'wdi_sh_xpd_oopc_pp_cd'],
 'wdi_ag_prd_food_xd': ['wdi_ag_prd_lvsk_xd'],
 'vdem_v2x_libdem': ['vdem_v2x_mpi',
  'vdem_v2x_polyarchy',
  'vdem_v2x_partipdem',
  'vdem_v2x_liberal',
  'vdem_v2xel_frefair',
  'vdem_v2xnp_pres',
  'vdem_v2x_neopat',
  'vdem_v2x_regime_amb',
  'vdem_v2xcl_rol',
  'vdem_v2x_veracc',
  'vdem_v2x_regime',
  'vdem_v2x_rule',
  'vdem_v2xcl_disc',
  'vdem_v2xlg_legcon',
  'vdem_v2xdl_delib']}